In [1]:
import warnings
warnings.filterwarnings("ignore")
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_histogram

## Zadanie 1
Zaimplementuj kwantowe rozwiązanie podstawowego problemu Deutcha dla jedno-qbitowej funkcji na wybranym symulatorze

Mamy daną jedną z czterech funkcji z tabelki 2, ale nie wiemy którą. 
Chcemy sprawdzić, czy jest ona stała czy zmienna. Ile razy musimy uruchomić tę funkcję, żeby to sprawdzić?

![](deutch_funkcje.png)

![](deutch_bramki.png)

![](deutch_test.png)

In [2]:
def f0(qc):
    pass

def f1(qc):
    qc.cx(control_qubit=0, target_qubit=1)

def f2(qc):
    qc.x(1)
    qc.cx(control_qubit=0, target_qubit=1)
    
def f3(qc):
    qc.x(1)
    
def add_h_all(qc, qnumber):
    for i in range(qnumber):
        qc.h(i)
    
def create_deutch(f):
    qnumber = 2
    qc = QuantumCircuit(qnumber)
    qc.initialize("10")
    add_h_all(qc, qnumber)
    f(qc)
    add_h_all(qc, qnumber)
    display(qc.draw())
    return qc

def test_statevector(qc):
    display(Statevector.from_instruction(qc).draw('latex'))

In [3]:
test_statevector(create_deutch(f0))

┌──────────────────┐┌───┐┌───┐
q_0: ┤0                 ├┤ H ├┤ H ├
     │  Initialize(1,0) │├───┤├───┤
q_1: ┤1                 ├┤ H ├┤ H ├
     └──────────────────┘└───┘└───┘

<IPython.core.display.Latex object>

In [4]:
test_statevector(create_deutch(f1))

┌──────────────────┐┌───┐     ┌───┐
q_0: ┤0                 ├┤ H ├──■──┤ H ├
     │  Initialize(1,0) │├───┤┌─┴─┐├───┤
q_1: ┤1                 ├┤ H ├┤ X ├┤ H ├
     └──────────────────┘└───┘└───┘└───┘

<IPython.core.display.Latex object>

In [5]:
test_statevector(create_deutch(f2))

┌──────────────────┐┌───┐          ┌───┐
q_0: ┤0                 ├┤ H ├───────■──┤ H ├
     │  Initialize(1,0) │├───┤┌───┐┌─┴─┐├───┤
q_1: ┤1                 ├┤ H ├┤ X ├┤ X ├┤ H ├
     └──────────────────┘└───┘└───┘└───┘└───┘

<IPython.core.display.Latex object>

In [6]:
test_statevector(create_deutch(f3))

┌──────────────────┐┌───┐┌───┐     
q_0: ┤0                 ├┤ H ├┤ H ├─────
     │  Initialize(1,0) │├───┤├───┤┌───┐
q_1: ┤1                 ├┤ H ├┤ X ├┤ H ├
     └──────────────────┘└───┘└───┘└───┘

<IPython.core.display.Latex object>

## Zadanie 2
Zaimplementuj rozwiązanie klasyczne oraz kwantowe  problemu Bernsteina-Vaziraniego na wybranym symulatorze.

Mamy daną funkcję f, ale nieznamy jej (ustalonego z góry) parametru a. 
Ile razy musimy uruchomić tę funkcję, żeby dowiedzieć się, ile wynosi a?

![](bernstein_vazirani_przyklady.png)

### Rozwiązanie klasyczne

![](bernstein_vazirani_klasyczne.png)

In [7]:
a = 0b01101
n = 5

def f(x, a):
    ax = 0
    for i in range(n):
        ax += (a % 2) * (x % 2) * 2 ** i
        a //= 2
        x //= 2
    return ax

def as_bin_str(x, n):
    return bin(x)[2:].zfill(n)

deduced_a = [0] * n
for i in range(0, n):
    x = 2 ** i
    ax = f(x, a)
    print(f'x:  {as_bin_str(x, n)}\nax: {as_bin_str(ax, n)}\n')
    deduced_a[n-i-1] = int(ax != 0)

a_str = as_bin_str(a, n)
deduced_a_str = ''.join(map(str, deduced_a))
print(f'a:         {a_str}\ndeduced_a: {deduced_a_str}\nequal:     {a_str == deduced_a_str}')

x:  00001
ax: 00001

x:  00010
ax: 00000

x:  00100
ax: 00100

x:  01000
ax: 01000

x:  10000
ax: 00000

a:         01101
deduced_a: 01101
equal:     True


**UWAGA**
Następne rozwiązanie oraz kwantowe są dla a = 11001, pierwsze dla a = 01101

In [13]:
n = 6

def create_bernstein_bazirani_classic(init_state):
    qnumber = n
    qc = QuantumCircuit(qnumber)
    qc.initialize(init_state)
    qc.cx(control_qubit=0, target_qubit=5)
    qc.cx(control_qubit=3, target_qubit=5)
    qc.cx(control_qubit=4, target_qubit=5)
    return qc

for i in range(0, n - 1):
    x = 2 ** i
    print(f'x:  {as_bin_str(x, n)}')
    test_statevector(create_bernstein_bazirani_classic(as_bin_str(x, n)))

x:  000001


<IPython.core.display.Latex object>

x:  000010


<IPython.core.display.Latex object>

x:  000100


<IPython.core.display.Latex object>

x:  001000


<IPython.core.display.Latex object>

x:  010000


<IPython.core.display.Latex object>

Patrzymy na rejestr wyjściowy i maskę x, która wskazuje gdzie bit z tego rejestru znajduje się w wyniku.
Otrzymujemy 11001 co zgadza się z zadanym argumentem

### Rozwiązanie kwantowe

![](bernstein_vazirani_kwantowe.png)

![](bernstein_vazirani_funkcja.png)

In [9]:
def create_bernstein_bazirani():
    qnumber = 6
    qc = QuantumCircuit(qnumber)
    qc.initialize('100000')
    add_h_all(qc, qnumber)
    qc.cx(control_qubit=0, target_qubit=5)
    qc.cx(control_qubit=3, target_qubit=5)
    qc.cx(control_qubit=4, target_qubit=5)
    add_h_all(qc, qnumber)
    display(qc.draw())
    return qc

test_statevector(create_bernstein_bazirani())

┌──────────────────────────┐┌───┐     ┌───┐          
q_0: ┤0                         ├┤ H ├──■──┤ H ├──────────
     │                          │├───┤  │  ├───┤          
q_1: ┤1                         ├┤ H ├──┼──┤ H ├──────────
     │                          │├───┤  │  ├───┤          
q_2: ┤2                         ├┤ H ├──┼──┤ H ├──────────
     │  Initialize(1,0,0,0,0,0) │├───┤  │  └───┘┌───┐     
q_3: ┤3                         ├┤ H ├──┼────■──┤ H ├─────
     │                          │├───┤  │    │  └───┘┌───┐
q_4: ┤4                         ├┤ H ├──┼────┼────■──┤ H ├
     │                          │├───┤┌─┴─┐┌─┴─┐┌─┴─┐├───┤
q_5: ┤5                         ├┤ H ├┤ X ├┤ X ├┤ X ├┤ H ├
     └──────────────────────────┘└───┘└───┘└───┘└───┘└───┘

<IPython.core.display.Latex object>

Wynik znajduje się na rejestrach wejściowych więc pomijamy pierwszy bit i otrzymujemy 11001, równe zadanemu argumentowi